In [ ]:
from huggingface_hub import login
login()

In [ ]:
from huggingface_hub import login
login("**KEY**")

In [ ]:
import gradio as gr
import torch
import easyocr
import clip
import numpy as np
from PIL import Image
from transformers import pipeline
import requests

# Hugging Face API details
API_KEY = "****KEY***"
API_URL = "https://api-inference.huggingface.co/models/tiiuae/falcon-7b-instruct"

def generate_ai_response(text, sentiment, category):
    print("\n🔹 Generating AI Response...")
    print(f"Text: {text}")
    print(f"Sentiment: {sentiment}")
    print(f"Category: {category}")

    prompt = (f"This meme contains the text: '{text}'.\n"
              f"It expresses a {sentiment.lower()} sentiment and belongs to the '{category}' category.\n"
              f"Provide a meaningful AI-generated response in a motivational way, avoiding personal pronouns like 'I' or 'me':")

    headers = {"Authorization": f"Bearer {API_KEY}"}
    payload = {"inputs": prompt, "parameters": {"max_new_tokens": 200, "temperature": 0.7, "top_p": 0.9, "repetition_penalty": 1.2}}

    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        print(f"API Response Code: {response.status_code}")

        if response.status_code == 200:
            generated_text = response.json()[0]["generated_text"][len(prompt):].strip()
            print(f"Generated AI Response: {generated_text}")
            return generated_text
        else:
            print(f"❌ API Error: {response.json()}")
            return f"❌ Error: {response.json()}"
    except Exception as e:
        print(f"Exception during API call: {e}")
        return "❌ API call failed"

# Load models
print("🔹 Loading models...")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)
print("✅ CLIP Model loaded successfully!")

reader = easyocr.Reader(['en'])
print("✅ EasyOCR Model loaded successfully!")

sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")
print("✅ Sentiment Analysis Model loaded successfully!")

# Define meme categories
positive_categories = ["funny", "wholesome", "motivational", "joyful", "victory", "cheerful"]
negative_categories = ["alone", "melancholic", "sad", "heartbroken", "depressed", "stress", "pain"]

keyword_to_category = {
    "alone": "alone", "melancholy": "melancholic", "sad": "sad", "heartbreak": "heartbroken",
    "depressed": "depressed", "pain": "pain", "hopeless": "hopeless", "anxious": "anxious",
    "funny": "funny", "happy": "joyful", "laugh": "funny", "love": "love", "inspire": "motivational",
    "cheer": "cheerful", "grateful": "grateful", "success": "victory"
}
import numpy as np
def analyze_meme(image):
    print("\n🔹 Processing Meme Image...")

    # Extract text using OCR

    image_np = np.array(image)  # Convert PIL Image to NumPy array
    extracted_text = " ".join(reader.readtext(image_np, detail=0))
    print(f"📜 Extracted Text: {extracted_text}")

    # Sentiment analysis
    sentiment_result = sentiment_pipeline(extracted_text)
    sentiment_label = sentiment_result[0]['label']
    print(f"🔹 Sentiment: {sentiment_label}")

    # Choose categories
    selected_categories = positive_categories if sentiment_label == "POSITIVE" else negative_categories
    print(f"🔹 Selected Categories: {selected_categories}")

    # Priority category based on text keywords
    priority_category = None
    for word, category in keyword_to_category.items():
        if word in extracted_text.lower():
            priority_category = category
            break
    print(f"🔹 Priority Category: {priority_category}")

    # Image classification using CLIP
    image_input = clip_preprocess(image).unsqueeze(0).to(device)
    category_texts = [clip.tokenize(category).to(device) for category in selected_categories]
    category_tensors = torch.cat(category_texts)

    with torch.no_grad():
        image_features = clip_model.encode_image(image_input)
        text_features = clip_model.encode_text(category_tensors)
        similarity_scores = (image_features @ text_features.T).softmax(dim=-1).cpu().numpy()

    print(f"📊 CLIP Similarity Scores: {similarity_scores}")

    # Select final category
    image_based_category = selected_categories[np.argmax(similarity_scores)]
    final_category = priority_category if priority_category else image_based_category
    confidence_score = 1.0 if priority_category else np.max(similarity_scores)

    print(f"✅ Final Category: {final_category} (Confidence: {confidence_score:.4f})")

    # AI-generated response
    ai_response = generate_ai_response(extracted_text, sentiment_label, final_category)

    return extracted_text, sentiment_label, final_category, f"{confidence_score:.4f}", ai_response

# Gradio Interface
print("🔹 Initializing Gradio Interface...")
iface = gr.Interface(
    fn=analyze_meme,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Textbox(label="Extracted Text"),
        gr.Textbox(label="Sentiment Type"),
        gr.Textbox(label="Category"),
        gr.Textbox(label="Confidence Score"),
        gr.Textbox(label="AI-Generated Response"),
    ],
    title="Meme Analyzer with AI Response",
    description="Upload a meme image to analyze its text, sentiment, category, and get an AI-generated response."
)


# Launch Gradio
print("🚀 Launching Gradio App...")
iface.launch(debug=True)


🔹 Loading models...
Using device: cuda
✅ CLIP Model loaded successfully!
✅ EasyOCR Model loaded successfully!


Device set to use cuda:0


✅ Sentiment Analysis Model loaded successfully!
🔹 Initializing Gradio Interface...
🚀 Launching Gradio App...
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d51eac975ef5814ecf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



🔹 Processing Meme Image...
📜 Extracted Text: It's Monday ALREADYQQ Seriously???
🔹 Sentiment: NEGATIVE
🔹 Selected Categories: ['alone', 'melancholic', 'sad', 'heartbroken', 'depressed', 'stress', 'pain']
🔹 Priority Category: None
📊 CLIP Similarity Scores: [[4.5156e-04 6.5029e-05 5.6427e-02 7.3193e-01 1.1224e-01 9.4543e-02
  4.4212e-03]]
✅ Final Category: heartbroken (Confidence: 0.7319)

🔹 Generating AI Response...
Text: It's Monday ALREADYQQ Seriously???
Sentiment: NEGATIVE
Category: heartbroken
API Response Code: 200
Generated AI Response: Monday can be tough, but let's power through it! Remember, you got this!
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7864 <> https://d51eac975ef5814ecf.gradio.live
